<a href="https://www.kaggle.com/code/rahulchauhan016/deepspace-ai-advanced-learning-platform-v2?scriptVersionId=296208443" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 🚀 DeepSpace AI - Advanced Learning Platform
## With Multi-Mode Prompt Engine

# ✅ 1 — Import Libraries

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, Image, HTML
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from enum import Enum
import json
from datetime import datetime

# ✅ 2 — Load Dataset

In [2]:
DATA_PATH = "/kaggle/input/nasa-astronomy-picture-of-the-day-1995-2026/nasa_apod_complete.csv"
df = pd.read_csv(DATA_PATH)
print("✅ Dataset loaded:", len(df), "records")
df.head()

✅ Dataset loaded: 11186 records


,date,explanation,hdurl,media_type,service_version,title,url,copyright
0,1995-06-16,Today's Picture: Explanation: If the Earth...,https://apod.nasa.gov/apod/image/e_lens.gif,image,v1,Neutron Star Earth,https://apod.nasa.gov/apod/image/e_lens.gif,NaN
1,1995-06-20,"Today's Picture: June 20, 1995 The Pleiades...",https://apod.nasa.gov/apod/image/pleiades2.gif,image,v1,Pleiades Star Cluster,https://apod.nasa.gov/apod/image/pleiades2.gif,NaN
2,1995-06-21,"Today's Picture: June 21, 1995 The Aftermat...",https://apod.nasa.gov/apod/image/sn1987a_hst.gif,image,v1,Supernova 1987a Aftermath,https://apod.nasa.gov/apod/image/sn1987a_hst.gif,NaN
3,1995-06-22,In 1972 Astronauts on the United States's last...,https://apod.nasa.gov/apod/image/earth_a17.gif,image,v1,Earth from Apollo 17,https://apod.nasa.gov/apod/image/earth_a17.gif,NaN
4,1995-06-23,"What if you could ""see"" gamma rays? This compu...",https://apod.nasa.gov/apod/image/egret_gro.gif,image,v1,Gamma Ray Sky Map,https://apod.nasa.gov/apod/image/egret_gro.gif,NaN


# ✅ 3 — Clean Dataset

In [3]:
# Lowercase column names
df.columns = df.columns.str.lower()

# Convert date
df["date"] = pd.to_datetime(df["date"])

# Keep only images
df = df[df["media_type"] == "image"]

# Remove missing explanations
df = df.dropna(subset=["explanation"])

# Remove duplicates
df = df.drop_duplicates(subset="title")

# Fill missing titles
df["title"] = df["title"].fillna("Unknown")

# Create searchable text
df["search_text"] = df["title"] + " " + df["explanation"]

print("✅ Clean rows:", len(df))

✅ Clean rows: 7732


# ✅ 4 — Build Smart Search Engine

In [4]:
vectorizer = TfidfVectorizer(
    stop_words="english",
    max_features=5000
)

tfidf_matrix = vectorizer.fit_transform(df["search_text"])
print("✅ Search engine ready!")

✅ Search engine ready!


# ✅ 5 — Find Best Match Function

In [5]:
def find_best_answer(question, df):
    q_vec = vectorizer.transform([question])
    similarity = cosine_similarity(q_vec, tfidf_matrix)
    best_idx = similarity[0].argsort()[::-1][0]
    
    return {
        "title": df.iloc[best_idx]["title"],
        "explanation": df.iloc[best_idx]["explanation"],
        "url": df.iloc[best_idx]["url"],
        "date": df.iloc[best_idx]["date"],
        "similarity": similarity[0][best_idx]
    }

print("✅ Search function ready!")

✅ Search function ready!


# ✅ 6 — Text Shortener

In [6]:
def short_answer(text, max_length=500):
    if len(text) <= max_length:
        return text
    
    sentences = text.split('.')
    result = ""
    
    for sentence in sentences:
        if len(result) + len(sentence) <= max_length:
            result += sentence + "."
        else:
            break
    
    return result if result else text[:max_length] + "..."

print("✅ Text shortener ready!")

✅ Text shortener ready!


# ✅ 7 — DeepSpace AI Prompt Modes Engine

In [7]:
# ===============================
# DeepSpace AI Prompt Modes
# ===============================

class PromptModes(Enum):
    BEGINNER = "beginner"
    FEYNMAN = "feynman"
    STUDENT = "student"
    RESEARCH = "research"
    DISCOVERY = "discovery"
    PLANETARIUM = "planetarium"
    LEARNING = "learning"
    EXPLORATION = "exploration"
    MISSION = "mission"

def beginner_mode(text):
    """Simplify to first 2 sentences - perfect for quick understanding"""
    parts = text.split(".")
    return " ".join(parts[:2]) + "."

def feynman_mode(text):
    """Explain as if to a 5-year-old - ultra simple"""
    parts = text.split(".")
    simple = " ".join(parts[:3])
    return "🎯 Simple Explanation:\n" + simple + "."

def student_mode(text):
    """Step-by-step learning format"""
    parts = text.split(".")
    steps = "\n".join([f"{i+1}. {part.strip()}" for i, part in enumerate(parts[:4]) if part.strip()])
    return "📚 Step-by-Step Learning:\n" + steps

def research_mode(text):
    """Detailed academic explanation"""
    parts = text.split(".")
    return "🔬 Research Details:\n" + " ".join(parts[:6]) + "."

def discovery_mode(title):
    """Suggest related discoveries"""
    return f"🔍 Discovery Exploration:\n\nRelated discoveries and observations connected to {title} can be explored in NASA APOD archives. Consider researching similar phenomena in the database!"

def planetarium_mode(title):
    """Help locate in sky"""
    return f"🌌 Planetarium Guide:\n\n{title} may be visible in certain seasons depending on your location. Use Stellarium or Skyview app to locate it in real-time. Best viewing times vary throughout the year!"

def learning_mode(title):
    """Structured lesson plan"""
    return f"""📖 Structured Lesson Plan:

1️⃣ Fundamentals: Understand what {title} is
2️⃣ Formation: Study how it forms and evolves
3️⃣ Observations: Explore real telescopic observations
4️⃣ Importance: Learn its significance in space science
5️⃣ Research: Investigate current scientific findings"""

def exploration_mode(title):
    """Suggest related topics"""
    return f"""🚀 Exploration Path:\n
If you enjoyed {title}, explore these related topics:
• Galaxies and galactic structures
• Nebulae and star formation regions
• Planetary systems and exoplanets
• Black holes and neutron stars
• Cosmological phenomena"""

def mission_mode(title):
    """Space missions connection"""
    return f"""🛰️ Space Missions Archive:

Several space missions have studied objects like {title}:
• NASA missions: Hubble, JWST, Chandra X-ray
• ESA missions: Euclid, Athena
• International collaborations

Explore NASA.gov and ESA.int for mission data and discoveries!"""

print("✅ All 9 Prompt Modes Loaded!")

✅ All 9 Prompt Modes Loaded!


# ✅ 8 — Main Assistant with Modes

In [8]:
def assistant_reply(question, mode="beginner"):
    """Main assistant function with mode selection"""
    
    result = find_best_answer(question, df)
    title = result["title"]
    explanation = result["explanation"]
    
    # Display header
    header = f"""
    <div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; color: white;'>
    <h2>🚀 DeepSpace AI Assistant</h2>
    <p><strong>Topic:</strong> {title}</p>
    <p><strong>Mode:</strong> {mode.upper()}</p>
    </div>
    """
    display(HTML(header))
    
    # Display image
    try:
        display(Image(url=result["url"]))
    except:
        print("⚠️ Image unavailable")
    
    print("\n" + "="*70 + "\n")
    
    # Apply mode
    if mode == "beginner":
        print(beginner_mode(explanation))
    elif mode == "feynman":
        print(feynman_mode(explanation))
    elif mode == "student":
        print(student_mode(explanation))
    elif mode == "research":
        print(research_mode(explanation))
    elif mode == "discovery":
        print(discovery_mode(title))
    elif mode == "planetarium":
        print(planetarium_mode(title))
    elif mode == "learning":
        print(learning_mode(title))
    elif mode == "exploration":
        print(exploration_mode(title))
    elif mode == "mission":
        print(mission_mode(title))
    else:
        print(beginner_mode(explanation))
    
    print("\n" + "="*70 + "\n")
    
    return {
        "title": title,
        "mode": mode,
        "timestamp": datetime.now().isoformat()
    }

print("✅ Assistant Ready!")

✅ Assistant Ready!


# ✅ 9 — Chat Memory System

In [9]:
chat_memory = []
learning_history = []

def save_to_history(title, mode):
    """Track learning progress"""
    learning_history.append({
        "title": title,
        "mode": mode,
        "timestamp": datetime.now().isoformat()
    })

def get_stats():
    """Show learning statistics"""
    if not learning_history:
        return "No history yet"
    
    unique_topics = len(set(h["title"] for h in learning_history))
    total_queries = len(learning_history)
    
    return f"""
    📊 Learning Statistics:
    • Total Queries: {total_queries}
    • Unique Topics: {unique_topics}
    • Topics Explored: {', '.join(set(h['title'] for h in learning_history[:5]))}
    """

print("✅ Memory system ready!")

✅ Memory system ready!


# ✅ 10 — Interactive Mode Examples

## Example 1: Beginner Mode

In [10]:
result = assistant_reply("What is a nebula?", mode="beginner")
save_to_history(result["title"], result["mode"])



Is the only thing unusual about this nebula its shape?  Pictured above is the IC 2220 - the Toby Jug Nebula - a reflection nebula surrounding a normal red giant star  Reflection nebulae shine by light reflected from the central star(s).




## Example 2: Feynman Mode

In [11]:
result = assistant_reply("Why is Mars red?", mode="feynman")
save_to_history(result["title"], result["mode"])



🎯 Simple Explanation:
When does Mars appear the largest? This occurs when Earth sweeps past Mars in their respective orbits around the Sun, creating a momentary Sun-Earth-Mars alignment called opposition  The featured image shows the Mars opposition that occurred earlier this year, as well as how Mars will look later this year   Mars actually changes its size continuously -- the monthly jumps in size are illustrative.




## Example 3: Student Mode

In [12]:
result = assistant_reply("How do stars form?", mode="student")
save_to_history(result["title"], result["mode"])



📚 Step-by-Step Learning:
1. Sometimes the unexpected comes in a familiar shape
2. In this picture, the seemingly familiar teardrop-shaped object just right of center is actually an unusually situated disk of gas and dust
3. In fact, the teardrop is about the size of our own Solar System and is racing against time to condense and form planets
4. This disk, however, is unfortunate enough to lie in the Trapezium, which is also home to several immense, bright stars




## Example 4: Learning Mode

In [13]:
result = assistant_reply("Saturn", mode="learning")
save_to_history(result["title"], result["mode"])



📖 Structured Lesson Plan:

1️⃣ Fundamentals: Understand what Looking Down on Saturn is
2️⃣ Formation: Study how it forms and evolves
3️⃣ Observations: Explore real telescopic observations
4️⃣ Importance: Learn its significance in space science
5️⃣ Research: Investigate current scientific findings




## Example 5: Research Mode

In [14]:
result = assistant_reply("Black hole formation", mode="research")
save_to_history(result["title"], result["mode"])



🔬 Research Details:
What does a black hole look like? To find out, radio telescopes from around the Earth coordinated observations of black holes with the largest known event horizons on the sky   Alone, black holes are just black, but these monster attractors are known to be surrounded by glowing gas   The first image was released yesterday and resolved the area around the black hole at the center of galaxy M87 on a scale below that expected for its event horizon   Pictured, the dark central region is not the event horizon, but rather the black hole's shadow -- the central region of emitting gas darkened by the central black hole's gravity  The size and shape of the shadow is determined by bright gas near the event horizon, by strong gravitational lensing deflections, and by the black hole's spin   In resolving this black hole's shadow, the Event Horizon Telescope (EHT) bolstered evidence that Einstein's gravity works even in extreme regions, and gave clear evidence that M87 has a c

## Learning Statistics

In [15]:
print(get_stats())


    📊 Learning Statistics:
    • Total Queries: 5
    • Unique Topics: 5
    • Topics Explored: First Horizon-Scale Image of a Black Hole, Looking Down on Saturn, The Toby Jug Nebula, Mars at Closest Approach, Trapezium: Teardrops in My Skies
    


# ✅ 11 — All 9 Modes Quick Reference

In [17]:
assistant_reply("Galaxies", mode="exploration")




🚀 Exploration Path:

If you enjoyed Two Million Galaxies, explore these related topics:
• Galaxies and galactic structures
• Nebulae and star formation regions
• Planetary systems and exoplanets
• Black holes and neutron stars
• Cosmological phenomena




{'title': 'Two Million Galaxies',
 'mode': 'exploration',
 'timestamp': '2026-02-06T11:10:33.002372'}

In [18]:
assistant_reply("Mars", mode="mission")



🛰️ Space Missions Archive:

Several space missions have studied objects like Mars at Closest Approach:
• NASA missions: Hubble, JWST, Chandra X-ray
• ESA missions: Euclid, Athena
• International collaborations

Explore NASA.gov and ESA.int for mission data and discoveries!




{'title': 'Mars at Closest Approach',
 'mode': 'mission',
 'timestamp': '2026-02-06T11:11:11.438199'}

In [19]:
assistant_reply("Nebulae", mode="discovery")




🔍 Discovery Exploration:

Related discoveries and observations connected to The Chameleon's Dark Nebulae can be explored in NASA APOD archives. Consider researching similar phenomena in the database!




{'title': "The Chameleon's Dark Nebulae",
 'mode': 'discovery',
 'timestamp': '2026-02-06T11:11:52.801904'}

In [20]:
assistant_reply("Black hole formation", mode="research")




🔬 Research Details:
What does a black hole look like? To find out, radio telescopes from around the Earth coordinated observations of black holes with the largest known event horizons on the sky   Alone, black holes are just black, but these monster attractors are known to be surrounded by glowing gas   The first image was released yesterday and resolved the area around the black hole at the center of galaxy M87 on a scale below that expected for its event horizon   Pictured, the dark central region is not the event horizon, but rather the black hole's shadow -- the central region of emitting gas darkened by the central black hole's gravity  The size and shape of the shadow is determined by bright gas near the event horizon, by strong gravitational lensing deflections, and by the black hole's spin   In resolving this black hole's shadow, the Event Horizon Telescope (EHT) bolstered evidence that Einstein's gravity works even in extreme regions, and gave clear evidence that M87 has a c

{'title': 'First Horizon-Scale Image of a Black Hole',
 'mode': 'research',
 'timestamp': '2026-02-06T11:12:26.339281'}

In [16]:
modes_info = """
🎯 DEEPSFACE AI - 9 LEARNING MODES
=====================================

✅ BEGINNER MODE
   Purpose: Ultra-simple 2-sentence summaries
   Best For: Quick understanding, first-time learners
   Example: assistant_reply("What is a nebula?", mode="beginner")

✅ FEYNMAN MODE
   Purpose: Explain like you're 5 years old
   Best For: Deep conceptual understanding
   Example: assistant_reply("Why is Mars red?", mode="feynman")

✅ STUDENT MODE
   Purpose: Step-by-step structured learning
   Best For: Homework, exams, deep dives
   Example: assistant_reply("How do stars form?", mode="student")

✅ RESEARCH MODE
   Purpose: Detailed academic explanation (6 sentences)
   Best For: Academic papers, presentations
   Example: assistant_reply("Black hole formation", mode="research")

✅ DISCOVERY MODE
   Purpose: Suggest related discoveries
   Best For: Exploration, finding new topics
   Example: assistant_reply("Nebulae", mode="discovery")

✅ PLANETARIUM MODE
   Purpose: How to locate in the night sky
   Best For: Stargazers, observers
   Example: assistant_reply("Andromeda Galaxy", mode="planetarium")

✅ LEARNING MODE
   Purpose: Structured 5-step lesson plan
   Best For: Teachers, complete learning
   Example: assistant_reply("Saturn rings", mode="learning")

✅ EXPLORATION MODE
   Purpose: Suggest related cosmic topics
   Best For: Curriculum building
   Example: assistant_reply("Galaxies", mode="exploration")

✅ MISSION MODE
   Purpose: Link to space missions studying the topic
   Best For: Understanding mission relevance
   Example: assistant_reply("Mars", mode="mission")

=====================================
TRY DIFFERENT MODES FOR SAME TOPIC!
"""

print(modes_info)


🎯 DEEPSFACE AI - 9 LEARNING MODES

✅ BEGINNER MODE
   Purpose: Ultra-simple 2-sentence summaries
   Best For: Quick understanding, first-time learners
   Example: assistant_reply("What is a nebula?", mode="beginner")

✅ FEYNMAN MODE
   Purpose: Explain like you're 5 years old
   Best For: Deep conceptual understanding
   Example: assistant_reply("Why is Mars red?", mode="feynman")

✅ STUDENT MODE
   Purpose: Step-by-step structured learning
   Best For: Homework, exams, deep dives
   Example: assistant_reply("How do stars form?", mode="student")

✅ RESEARCH MODE
   Purpose: Detailed academic explanation (6 sentences)
   Best For: Academic papers, presentations
   Example: assistant_reply("Black hole formation", mode="research")

✅ DISCOVERY MODE
   Purpose: Suggest related discoveries
   Best For: Exploration, finding new topics
   Example: assistant_reply("Nebulae", mode="discovery")

✅ PLANETARIUM MODE
   Purpose: How to locate in the night sky
   Best For: Stargazers, observers
   